# CODA-database-sandbox

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Date: 11 September 2024

# detect compute platform

In [1]:
import os
import socket
    
def get_platform():
    hostname = socket.gethostname()
    if "gadi" in hostname:  # Adjust this condition to fit your HPC's hostname or unique identifier
        return 'HPC',hostname
    else:
        return 'Laptop',hostname
[platform,hostname] = get_platform()
print('the platform we are working on is '+platform+' with hostname: '+hostname)

the platform we are working on is HPC with hostname: gadi-cpu-bdw-0122.gadi.nci.org.au


### import packages

In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33909,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:36631,Total threads: 4
Dashboard: /proxy/38413/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:39973,


In [4]:
if platform == 'HPC':
    data_path = '/g/data/es60/users/thomas_moore/CODA/2005/'
    write_path = '/g/data/es60/users/thomas_moore/CODA/parquet/'
else:
    data_path = '/Users/moo270/data/CARSv2/CODA/'
    write_path = '/Users/moo270/data/CARSv2/CODA/parquet/'

print(f"Using data path: {data_path}")
print(f"Using write path: {write_path}")

Using data path: /g/data/es60/users/thomas_moore/CODA/2005/
Using write path: /g/data/es60/users/thomas_moore/CODA/parquet/


# build PQ & CSV versions from NetCDF

In [5]:
ds = xr.open_dataset(data_path+ "WOD_CODA_2005_pfl.nc",chunks="auto")

In [6]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates:
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
Data variables: (12/29)
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    country                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    dataset                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Access_no               (cast) float64 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    Recorder                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    crs                     int32 4B ...
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

In [7]:
ds = ds.set_coords(['WOD_id',
                    'origflagset',
                    'country',
                    'dataset',
                    'Access_no',
                    'dbase_orig',
                    'Project',
                    'WOD_cruise_identifier',
                    'Institute',
                    'Ocean_Vehicle',
                    'Temperature_Instrument',
                    'CODA_id'])

In [8]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates: (12/18)
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Institute               (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Ocean_Vehicle           (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Temperature_Instrument  (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Data variables: (12/17)
    Recorder                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Platform                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Temperature             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Temperature_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Temperature_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Salinity                (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    ...                      ...
    Chlorophyll             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    crs                     int32 4B ...
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

# make truncated version of ds

In [30]:
ds_truncated = ds#.isel(cast=slice(0,10000))

In [31]:
%%time
df = ds_truncated[['Temperature']].to_dataframe().reset_index()
df

CPU times: user 1min 48s, sys: 48.4 s, total: 2min 36s
Wall time: 2min 45s


,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',4.473917,b'WODpfl200501010001'
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',9.444819,b'WODpfl200501010001'
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',13.918531,b'WODpfl200501010001'
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',18.889204,b'WODpfl200501010001'
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',24.257399,b'WODpfl200501010001'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64420495,64100,1000,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420496,64100,1001,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420497,64100,1002,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420498,64100,1003,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'


# Try removing all-column-NaN-rows here? - https://github.com/CARSv2/cars-v2/issues/26

In [32]:
%%time
df1 = df.dropna(subset=['Temperature', 'z']).reset_index(drop=True)

CPU times: user 2.27 s, sys: 692 ms, total: 2.97 s
Wall time: 2.83 s


In [33]:
df1

,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',4.473917,b'WODpfl200501010001'
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',9.444819,b'WODpfl200501010001'
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',13.918531,b'WODpfl200501010001'
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',18.889204,b'WODpfl200501010001'
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',24.257399,b'WODpfl200501010001'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294676,64100,44,0.474000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',613.075928,b'WODpfl200512310207'
4294677,64100,45,0.463000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',632.863708,b'WODpfl200512310207'
4294678,64100,46,0.448000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',652.847534,b'WODpfl200512310207'
4294679,64100,47,0.438000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',672.532593,b'WODpfl200512310207'


In [34]:
%%time
df2 = df.dropna(subset=['z']).reset_index(drop=True)

CPU times: user 2.16 s, sys: 621 ms, total: 2.78 s
Wall time: 2.98 s


In [35]:
df2

,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',4.473917,b'WODpfl200501010001'
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',9.444819,b'WODpfl200501010001'
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',13.918531,b'WODpfl200501010001'
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',18.889204,b'WODpfl200501010001'
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',24.257399,b'WODpfl200501010001'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4298224,64100,44,0.474000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',613.075928,b'WODpfl200512310207'
4298225,64100,45,0.463000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',632.863708,b'WODpfl200512310207'
4298226,64100,46,0.448000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',652.847534,b'WODpfl200512310207'
4298227,64100,47,0.438000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',672.532593,b'WODpfl200512310207'


In [36]:
%%time
df3 = df.dropna(subset=['Temperature']).reset_index(drop=True)

CPU times: user 3.41 s, sys: 1.21 s, total: 4.62 s
Wall time: 4.91 s


In [37]:
df3

,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',4.473917,b'WODpfl200501010001'
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',9.444819,b'WODpfl200501010001'
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',13.918531,b'WODpfl200501010001'
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',18.889204,b'WODpfl200501010001'
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',24.257399,b'WODpfl200501010001'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294676,64100,44,0.474000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',613.075928,b'WODpfl200512310207'
4294677,64100,45,0.463000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',632.863708,b'WODpfl200512310207'
4294678,64100,46,0.448000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',652.847534,b'WODpfl200512310207'
4294679,64100,47,0.438000,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',672.532593,b'WODpfl200512310207'


# FIX THIS ???? go from DS - DDF lazily

# convert to dask dataframe

In [ ]:
%%time
ddf = dd.from_pandas(df, npartitions=500)

In [ ]:
ddf

# find columns with byte strings and convert ( is this needed when going from ds --> df --> ddf )

In [ ]:
ddf.head()

In [ ]:
%%time
ddf.info(memory_usage='deep')

In [ ]:
%%time
ddf.to_parquet(write_path+"2005_pfl_temp.pq",engine='pyarrow', compression='snappy')

In [ ]:
%%time
df.to_csv(write_path+"2005_pfl_temp.csv", index=False)

# load parquet data & filter

In [ ]:
%%time
temp_pfl_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")
# Search for rows where 'column_name' matches a condition
result_parquet = temp_pfl_ddf[temp_pfl_ddf ['z'] >= 2000].compute()

In [ ]:
result_parquet

# load CSV data and filter

In [ ]:
%%time
temp_pfl_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks
# Search for rows where 'column_name' matches a condition
result_csv = temp_pfl_csv_ddf[temp_pfl_csv_ddf['z'] >= 2000].compute()

# lazy load temperature

In [ ]:
%%time
temp_pq_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")

In [ ]:
%%time
temp_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks

In [ ]:
%%time
temp_csv_ddf[temp_csv_ddf['z'] >= 2000].compute()

In [ ]:
%%time
temp_pq_ddf[temp_pq_ddf['z'] >= 2000].compute()

## Polars

In [ ]:
import polars as pl

In [ ]:
%%time
# Load a large dataset with Dask (e.g., CSV)
dask_df = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)

# Convert Dask DataFrame to Pandas
pandas_df = dask_df.compute()

# Convert Pandas DataFrame to Polars
polars_df = pl.from_pandas(pandas_df)

In [ ]:
%%time
polars_df[polars_df['z'] >= 2000].compute()

# list of todo's

- create parquet from a df
- consider utility of turning "metadata" vars into coords
- consider removing NaN's at df step
- consider removing NaN's otherwise



In [ ]:
df.dropna(how='all',column='z')

In [ ]:
%%time
df = ds.to_dataframe()#.reset_index()